In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
DF = pd.read_csv("train.csv")
print(DF.shape)
DF[:10]

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [3]:
Y = np.array(DF['Survived'])

In [4]:
names = []
for x in np.array(DF["Name"]):
    tokens = x.split('. ', maxsplit = 1)
    names.append(tokens)
namedata = pd.DataFrame(names, columns = ["Surname/title", "First/Middle"])

In [5]:
namedata[:5]

,Surname/title,First/Middle
0,"Braund, Mr",Owen Harris
1,"Cumings, Mrs",John Bradley (Florence Briggs Thayer)
2,"Heikkinen, Miss",Laina
3,"Futrelle, Mrs",Jacques Heath (Lily May Peel)
4,"Allen, Mr",William Henry


In [6]:
surnamesandtitles = pd.DataFrame(namedata["Surname/title"])
names2 = []
for x in np.array(surnamesandtitles["Surname/title"]):
    tokens = x.split(', ', maxsplit = 1)
    names2.append(tokens)
names2 = pd.DataFrame(names2, columns = ["Surname", "Title"])
np.unique(names2["Title"])
DF = pd.concat([DF, names2], axis = 1)

In [7]:
DF[:5]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Surname,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Braund,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Cumings,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Heikkinen,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Futrelle,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Allen,Mr


In [8]:
for x in np.unique(DF["Title"]):
    tokenDF = pd.DataFrame(DF[DF["Title"]==x])
    averageage = np.mean(tokenDF["Age"])
    empties = tokenDF[tokenDF["Age"].isnull()]
    DF[(DF["Title"]==x) & (DF["Age"].isnull())] = DF[(DF["Title"]==x) & (DF["Age"].isnull())].fillna(value = averageage)

In [9]:
from sklearn.neural_network import MLPClassifier as ANN

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.preprocessing import normalize
import warnings
warnings.filterwarnings("error")

In [23]:
data = pd.DataFrame(DF[['Pclass', 'Sex', 'Age', 'Fare']])
data['Sex']= data['Sex'].astype('category')
data['Sex'] = data['Sex'].cat.codes
normal_age = pd.DataFrame(normalize(np.array([data['Age']])).reshape(-1,1), columns = ['Age_normalized'])
normal_fare = pd.DataFrame(normalize(np.array([data['Fare']])).reshape(-1,1), columns = ['Fare_normalized'])
X = pd.DataFrame(data[['Pclass', 'Sex']])
X = pd.concat([X, normal_fare, normal_age], axis = 1)
X.head()

,Pclass,Sex,Fare_normalized,Age_normalized
0,3,1,0.004103,0.022622
1,1,0,0.040344,0.039075
2,3,0,0.004485,0.026736
3,1,0,0.030053,0.035990
4,3,1,0.004556,0.035990


In [29]:
errs=[]
nsplits=100
X = np.array(X)

for j in range(nsplits):
    XTRAIN, XTEST, YTRAIN, YTEST=train_test_split(X,Y, test_size = .2)
    r = ANN(solver = 'lbfgs', alpha=1e-5, hidden_layer_sizes=(3,4,4), random_state = 1)
    r.fit(XTRAIN,YTRAIN)
    YP=r.predict(XTEST)
    errs.append(1-accuracy_score(YTEST,YP))
    
print("LDA mean error=%7.5f std=%7.5f" %(np.mean(errs),np.std(errs)))

LDA mean error=0.20564 std=0.03063


In [30]:
testDF = pd.read_csv("test.csv")

In [31]:
testnames = []
for x in np.array(testDF["Name"]):
    tokens = x.split('. ', maxsplit = 1)
    testnames.append(tokens)
testnamedata = pd.DataFrame(testnames, columns = ["Surname/title", "First/Middle"])

testsurnamesandtitles = pd.DataFrame(testnamedata["Surname/title"])
testnames2 = []
for x in np.array(testsurnamesandtitles["Surname/title"]):
    tokens = x.split(', ', maxsplit = 1)
    testnames2.append(tokens)
testnames2 = pd.DataFrame(testnames2, columns = ["Surname", "Title"])
np.unique(testnames2["Title"])
testDF = pd.concat([testDF, testnames2], axis = 1)

In [32]:
testDF[:5]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Surname,Title
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,Kelly,Mr
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,Wilkes,Mrs
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,Myles,Mr
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,Wirz,Mr
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,Hirvonen,Mrs


In [33]:
for x in np.unique(testDF["Title"]):
    tokenDF = pd.DataFrame(testDF[testDF["Title"]==x])
    averageage = np.mean(tokenDF["Age"])
    empties = tokenDF[tokenDF["Age"].isnull()]
    testDF[(testDF["Title"]==x) & (testDF["Age"].isnull())] = testDF[(testDF["Title"]==x) & (testDF["Age"].isnull())].fillna(value = averageage)

In [34]:
testDF[(testDF["Title"]=="Ms") & (testDF["Age"].isnull())] = testDF[(testDF["Title"]=="Ms") & (testDF["Age"].isnull())].fillna(value = 21.774844)
testDF[:5]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Surname,Title
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,Kelly,Mr
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,Wilkes,Mrs
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,Myles,Mr
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,Wirz,Mr
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,Hirvonen,Mrs


In [39]:
testdata = pd.DataFrame(testDF[['Pclass', 'Sex', 'Age', 'Fare']])
testdata['Sex']= testdata['Sex'].astype('category')
testdata['Sex'] = testdata['Sex'].cat.codes

thirdclass = testdata[testdata['Pclass']==3]
testdata[testdata['Fare'].isnull()] = testdata[testdata['Fare'].isnull()].fillna(value = np.mean(thirdclass['Fare']))

test_normal_age = pd.DataFrame(normalize(np.array([testdata['Age']])).reshape(-1,1), columns = ['Age_normalized'])
test_normal_fare = pd.DataFrame(normalize(np.array([testdata['Fare']])).reshape(-1,1), columns = ['Fare_normalized'])
XT = pd.DataFrame(testdata[['Pclass', 'Sex']])
XT = pd.concat([XT, test_normal_fare, test_normal_age], axis = 1)
XT.head()

,Pclass,Sex,Fare_normalized,Age_normalized
0,3,1,0.005788,0.051335
1,3,0,0.005175,0.069935
2,2,1,0.007162,0.092255
3,3,1,0.006404,0.040175
4,3,0,0.009084,0.032736


In [40]:
YT = r.predict(XT)

In [41]:
pass_id = testDF["PassengerId"]
output = pd.DataFrame({"PassengerId":np.array(pass_id),
                  "Survived": YT})
output.to_csv("result_neural_nets.csv", sep =",",index = False)

In [42]:
output.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
